In [1]:
# !pip uninstall mxnet-cu101
!pip install mxnet-cu101
# !pip uninstall mxnet
# !pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers==2.1.1
!pip install torch==1.3.1

     |████████████████████████████████| 710.5MB 24kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 256kB 4.7MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp36-cp36m-linux_x86_64.whl size=470034 sha256=6960bf071d602ef277237dc68f4bb5078e8178d7fe4edfc2a69b3bba94a1cb46
  Stored in directory: /root/.cache/pip/wheels/af/60/16/1f8a40e68b85bd9bd7960e91830bca5e40cd113f3220b7e231
Successfully built gluonnlp
     |████████████████████████████████| 1.0MB 4.1MB/s 
     |████████████████████████████████| 317kB 4.5MB/s 
     |████████████████████████████████| 890kB 12.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=6650f654a39c21e124b8177c936e19453d02f8a8ac641a30eb1c1499fa4147fe
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremo

# 새 섹션

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-2u61nh5k
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-2u61nh5k
  Created wheel for kobert: filename=kobert-0.1.1-cp36-none-any.whl size=12871 sha256=76aa03c596806f8f260a9ca5d42496cf46c1dc8fc8e8b803bfe27dc31eccbf12
  Stored in directory: /tmp/pip-ephem-wheel-cache-p5dngx85/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split

In [4]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [5]:
from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule

In [6]:
##GPU 사용 시
device = torch.device("cuda:0")

In [7]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [8]:
# !wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
# !wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

In [9]:
cd /content/drive/My Drive/colab/KoBERT

/content/drive/My Drive/colab/KoBERT


In [10]:
# # total_data = pd.concat([pd.read_csv("backup/sentence_big_train.txt", sep='\t', index_col = 'idx'), pd.read_csv("backup/sentence_big_test.txt", sep='\t', index_col = 'idx')])
# total_data = pd.read_csv("backup/new_sentences.txt", sep='\t', index_col='idx')
# train_ratio, val_ratio = 0.7, 0.2
# test_num = 100
# data_size = len(total_data)
# train = total_data[:int(train_ratio*data_size)]
# val = total_data[int(train_ratio*data_size):int((train_ratio + val_ratio)*data_size)]
# test = total_data[int((train_ratio + val_ratio)*data_size):int((train_ratio + val_ratio)*data_size)+test_num]


# train.to_csv('data/train.txt', sep='\t', index_label='idx')
# val.to_csv('data/val.txt', sep='\t', index_label='idx')
# test.to_csv('data/test.txt', sep='\t', index_label='idx')

In [11]:
label_dic = dict([(0,'anger'), (1,'fear'), (2,'happiness'), (3,'longing'), (4,'sadness'), (5,'surprised'), (6, 'worry') ])

total_data = pd.read_csv("backup/new_sentences2.txt", sep='\t', index_col='idx')
total_data['sentiment'] = total_data['sentiment'].apply(lambda x : list(label_dic.keys())[list(label_dic.values()).index(x)])
train_ratio, val_ratio = 0.7, 0.3
data_size = len(total_data)
train, val = train_test_split(total_data, test_size=val_ratio, random_state=77)

train.to_csv('data/train.txt', sep='\t', index_label='idx')
val.to_csv('data/val.txt', sep='\t', index_label='idx')

In [12]:
# dataset_train = nlp.data.TSVDataset('data/train.txt', field_indices=[1,2], num_discard_samples=1)
# dataset_val = nlp.data.TSVDataset('data/val.txt', field_indices=[1,2], num_discard_samples=1)
# dataset_test = nlp.data.TSVDataset('data/test.txt', field_indices=[1,2], num_discard_samples=1)

In [13]:
dataset_train = nlp.data.TSVDataset('data/train.txt', field_indices=[1,2], num_discard_samples=1)
dataset_val = nlp.data.TSVDataset('data/val.txt', field_indices=[1,2], num_discard_samples=1)

In [14]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [15]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [16]:
## Setting parameters
max_len = 64
batch_size = 128
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [17]:
# data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
# data_val = BERTDataset(dataset_val, 0, 1, tok, max_len, True, False)
# data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [18]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_val = BERTDataset(dataset_val, 0, 1, tok, max_len, True, False)

In [19]:
# train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
# val_dataloader = torch.utils.data.DataLoader(data_val, batch_size=batch_size, num_workers=5)
# test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=len(data_test), num_workers=5)

In [20]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
val_dataloader = torch.utils.data.DataLoader(data_val, batch_size=batch_size, num_workers=5)

In [21]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        # self.classifier = nn.Sequential(
        #     nn.Linear(hidden_size, num_classes),
        #     nn.LogSoftmax()
        # )

        self.softmax = nn.LogSoftmax()
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        
        out = self.classifier(out)
        return self.softmax(out)
        # return self.classifier(out)

    def proba(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.softmax(out)

In [22]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [23]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [24]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.NLLLoss()

In [25]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [26]:
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total)

In [27]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [29]:
version = ((time.localtime()[2]), (time.localtime()[3]+9) % 24, (time.localtime()[4]))
n_val_loss = 10000000


for e in range(num_epochs):
    mylist = []
    train_acc = 0.0
    val_acc = 0.0
    test_acc = 0.0
    total_loss = 0.0
    total_val_loss = 0.0
    model.train()                            
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()

        # forward
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        label = label.long().to(device)
        valid_length= valid_length

        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)

        # backward
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        # accuracy
        train_acc += calc_accuracy(out, label)

        # loss
        total_loss += loss.item()

        # print
        if batch_id % log_interval == 0:
            print("Epoch {}, Batch id {}, Train Acc {}, Train Loss: {:.4f}".format(e+1, batch_id, train_acc / (batch_id+1), loss.data.cpu().numpy()))

    train_loss = total_loss / len(train_dataloader)
    print("Epoch {}, Train Acc: {}, Train Loss: {:.4f}".format(e+1, train_acc / (batch_id+1), train_loss))

    # evaluation
    # validation 데이터를 부르고 epoch 마다 학습된 모델을 부르고 평가.
    model.eval()
    with torch.no_grad():
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(val_dataloader):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            label = label.long().to(device)
            valid_length= valid_length
            
            out = model(token_ids, valid_length, segment_ids)
            loss_val = loss_fn(out, label)
            
            # val accuracy
            val_acc += calc_accuracy(out, label)

            # val loss
            total_val_loss += loss_val.item()

    val_loss = total_val_loss / len(val_dataloader)
    print ('Epoch {}, Eval Acc: {}, Eval Loss: {:.4f}'.format(e+1, val_acc / len(val_dataloader), val_loss))

    # Best Model 저장
    if val_loss < n_val_loss:
        n_val_loss = val_loss
        torch.save(model.state_dict(), f'/content/drive/My Drive/colab/KoBERT/saved_model/model_{version}.pth')
        print("")
        print("Train loss :" + str(train_loss))
        print("Validation loss :" + str(val_loss))
        print("Best Model saved......")
        print("----------------------------------------------------")


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 1, Batch id 0, Train Acc 0.09375, Train Loss: 2.0639


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-29-af35eb2cd9ec>", line 28, in <module>
    optimizer.step()
  File "/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py", line 51, in wrapper
    return wrapped(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/transformers/optimization.py", line 167, in step
    exp_avg_sq.mul_(beta2).addcmul_(1.0 - beta2, grad, grad)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (

KeyboardInterrupt: ignored

# Document Sentiment Analysis

In [28]:
 !pip install kss
import kss
import numexpr as ne

  Created wheel for kss: filename=kss-1.3.1-cp36-cp36m-linux_x86_64.whl size=251579 sha256=7156ba29e8499df7a4523f05a3ba15aaa2a17767aa31118631d272a707c66e6b
  Stored in directory: /root/.cache/pip/wheels/8b/98/d1/53f75f89925cd95779824778725ee3fa36e7aa55ed26ad54a8
Successfully built kss


In [ ]:
# essays = pd.read_csv("/content/drive/My Drive/colab/KoBERT/data/brunch crawling_25129.csv", index_col='Unnamed: 0')
# essays.dropna(axis = 0, inplace=True)
# essays['label'] = -1
# essays['label_'] = ''
# essays.reset_index(drop=True,inplace=True)
# essays.to_csv('/content/drive/My Drive/colab/KoBERT/data/essays.csv', index_label = 'idx')

In [29]:
import pandas as pd
essays = pd.read_csv('data/essays.csv', index_col = 'idx')

In [30]:
essays.label_.value_counts()

happiness    14674
sadness       2849
surprised     2822
longing       1942
anger         1356
fear           959
worry          324
Name: label_, dtype: int64

In [31]:
test_model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
# weights = torch.load(f'saved_model/model_{version}.pth', map_location='cuda:0')
weights = torch.load('saved_model/model_(15, 19, 49).pth', map_location='cuda:0')
test_model.load_state_dict(weights)
test_model = test_model.to(device)
test_model.eval()
print("Model Loaded")

Model Loaded


In [32]:
label_dic = dict([(0,'anger'), (1,'fear'), (2,'happiness'), (3,'longing'), (4,'sadness'), (5,'surprised'), (6, 'worry') ])

In [105]:
def Document_sentiment(document, model):
  essay = pd.DataFrame(kss.split_sentences(document))
  essay['label'] = 1
  essay.to_csv('data/sentences.txt', sep='\t', index_label='idx')
  dataset_sentences = nlp.data.TSVDataset('data/sentences.txt', field_indices=[1,2], num_discard_samples=1)
  data_sentences = BERTDataset(dataset_sentences, 0, 1, tok, max_len, True, False)
  sentences_dataloader = torch.utils.data.DataLoader(data_sentences, batch_size=len(data_sentences), num_workers=5)

  with torch.no_grad():
      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(sentences_dataloader):
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)
          label = label.long().to(device)
          valid_length= valid_length            
          outputs = model(token_ids, valid_length, segment_ids)
          
  pred_test = outputs
  arr = np.array(pred_test.tolist())
  arr = ne.evaluate("exp(arr)") #exponential
  for i in range(7):
    essay[label_dic[i]] = [proba[i] for proba in arr]
  essay['label'] = list(map(np.argmax, arr)) 
  indices = np.array(list(map(np.max, arr))).argsort()[::-1][0:min(len(essay),10)]
  prob = essay.iloc[indices].sum(axis=0)[2:].astype(float)

  # 이건 임의로 고칩시다.
  prob['happiness'] *= 0.6
  prob['fear'] *= 0.8
  prob['worry'] *= 2
  #
  
  result = prob.idxmax()
  return result

In [ ]:
label_count = dict([('anger', 0), ('fear', 0), ('happiness', 0), ('longing', 0), ('sadness', 0), ('surprised', 0), ('worry', 0) ])
for i in tqdm_notebook(range(len(essays))):
  # if essays['label'][i] != 1:
  #   continue
  result = Document_sentiment(essays['main_text'][i], test_model)
  essays['label'][i] = result 
  label_count[result] += 1
  if i % 20 == 0:
    print(label_count)
  if i % 500 == 0:
    essays.to_csv('data/essays.csv', index_label = 'idx')

Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable


{'anger': 1566, 'fear': 753, 'happiness': 6886, 'longing': 1811, 'sadness': 2190, 'surprised': 2329, 'worry': 886}


Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable


{'anger': 1568, 'fear': 755, 'happiness': 6894, 'longing': 1812, 'sadness': 2192, 'surprised': 2334, 'worry': 886}


Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable


...

Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable


{'anger': 1621, 'fear': 784, 'happiness': 7151, 'longing': 1881, 'sadness': 2288, 'surprised': 2421, 'worry': 915}


Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable


{'anger': 1622, 'fear': 786, 'happiness': 7158, 'longing': 1885, 'sadness': 2289, 'surprised': 2425, 'worry': 916}


Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable
Exception ignored when trying to write to the signal wakeup fd:
BlockingIOError: [Errno 11] Resource temporarily unavailable


In [ ]:
 essays.to_csv('data/essays.csv', index_label = 'idx', encoding='UTF-8')